# W State Benchmark

In [ ]:
import sys, pathlib
sys.path.append(str(pathlib.Path('..').resolve().parent))
from benchmarks.circuits import w_state_circuit
from benchmarks.backends import (
    StatevectorAdapter, StimAdapter, MPSAdapter,
)
from benchmarks.runner import BenchmarkRunner
from quasar import SimulationEngine
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

qubit_counts = range(2, 13, 2)

backends = [StatevectorAdapter(), StimAdapter(), MPSAdapter(), DecisionDiagramAdapter()]

runner = BenchmarkRunner()
engine = SimulationEngine()

for n in qubit_counts:
    circ = w_state_circuit(n)
    plan = engine.planner.plan(circ)
    backend = plan.steps[0].backend.name if plan.steps else None
    print(f"{n} qubits -> {backend}")
    for b in backends:
        try:
            rec = runner.run(circ, b, return_state=False)
            rec['qubits'] = n
            rec['selected_backend'] = None
        except Exception:
            continue
    try:
        rec = runner.run_quasar(circ, engine)
        rec['qubits'] = n
        rec['selected_backend'] = backend
    except Exception:
        runner.results.append({'framework':'quasar','time':float('nan'),'result':None,'qubits':n,'selected_backend':backend})

df = pd.DataFrame(runner.results)
print(df[df['framework']=='quasar'][['qubits','selected_backend']])

sns.lineplot(data=df, x='qubits', y='time', hue='framework')
plt.yscale('log')
plt.show()
